In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import time
from collections import deque
from matplotlib import animation

In [ ]:
maze = [
    ['S', 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 'G'],
]


In [ ]:
def to_numeric_grid(maze):
    mapping = {'S': 2, 'G': 3}
    numeric_maze = np.array([[mapping.get(cell, cell) for cell in row] for row in maze], dtype=int)
    return numeric_maze

maze = to_numeric_grid(maze)


In [ ]:
def find_pos(maze, value):
    for r, row in enumerate(maze):
        for c, val in enumerate(row):
            if val == value:
                return (r, c)

print(find_pos(maze, 0))


In [ ]:
def show_maze(maze):
    cmap = mcolors.ListedColormap(['white', 'black', 'orange', 'red'])
    plt.imshow(maze, cmap=cmap)
    plt.axis('off')
    plt.title("Unsolved Maze")
    plt.show()

show_maze(maze)


In [ ]:
def get_neighbors(r, c, maze):
    directions = [(-1,0), (1,0), (0,-1), (0,1)]
    for dr, dc in directions:
        nr, nc = r + dr, c + dc
        if 0 <= nr < maze.shape[0] and 0 <= nc < maze.shape[1] and maze[nr][nc] != 1:
            yield (nr, nc)

get_neighbors(4,5,maze)

In [ ]:
def solve_maze(maze, algorithm='bfs'):
    maze = to_numeric_grid(maze)
    start = find_pos(maze, 2)
    goal = find_pos(maze, 3)

    visited = set()
    steps = 0
    frontier = deque([(start, [start])]) if algorithm == 'bfs' else [(start, [start])]

    fig, ax = plt.subplots()
    cmap = mcolors.ListedColormap(['white', 'black', 'orange', 'red', 'green', 'blue'])  # 0-5
    display_maze = maze.copy()
    images = []

    start_time = time.time()

    found = False
    final_path = []

    while frontier:
        (r, c), current_path = frontier.popleft() if algorithm == 'bfs' else frontier.pop()
        steps += 1

        if (r, c) in visited:
            continue
        visited.add((r, c))

        if maze[r][c] not in [2, 3]:
            display_maze[r][c] = 4  # Green (visited)

        images.append([ax.imshow(display_maze.copy(), cmap=cmap, animated=True)])

        if (r, c) == goal:
            for pr, pc in current_path:
                if display_maze[pr][pc] not in [2, 3]:
                    display_maze[pr][pc] = 5  # Blue (final path)
            images.append([ax.imshow(display_maze.copy(), cmap=cmap, animated=True)])
            found = True
            final_path = current_path
            break

        for nr, nc in get_neighbors(r, c, maze):
            if (nr, nc) not in visited:
                frontier.append(((nr, nc), current_path + [(nr, nc)]))

    end_time = time.time()

    ani = animation.ArtistAnimation(fig, images, interval=50, repeat=False)
    plt.title(f"{algorithm.upper()} Maze Solver")
    plt.axis('off')

    def on_key(event):
        if event.key.lower() == 'q':
            plt.close(fig)

    fig.canvas.mpl_connect('key_press_event', on_key)
    plt.show()

    if found:
        print(f"\nPath found using {algorithm.upper()}")
        print(f"Path length: {len(final_path)}")
    else:
        print(f"No path found using {algorithm.upper()}")
        print(f"Path length: 0")

    print(f"Steps taken: {steps}")
    print(f"Visited nodes: {len(visited)}")
    print(f"Execution time: {end_time - start_time:.4f} seconds")


In [ ]:
def run():
    algo = input("Choose algorithm (BFS/DFS): ").lower()
    if algo not in ['bfs', 'dfs']:
        print("Invalid option.")
        return
    solve_maze(maze, algorithm=algo)

run()